In [1]:
import tensorflow as tf
import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from conversion_tf import GEMMDecisionTreeImplEdgeTPU, GEMMDecisionTreeImpl
from hummingbird.ml import convert

2023-02-09 18:43:19.912063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x = tf.constant([5, 5, 5, 5, 5, 5, 5, 5], shape=[1,8])
y = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1,8])

2023-02-09 18:43:21.817737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 18:43:21.818322: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
A = tf.constant([[1, 1, 1], [2, 2, 2], [3, 3, 3]], shape=[3,3])
B = tf.constant([[1, 2, 3], [1, 2, 3], [1, 2, 3]], shape=[3,3])

A_t = torch.tensor([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
B_t = torch.tensor([[1, 2, 3], [1, 2, 3], [1, 2, 3]])

In [4]:
tf.multiply(A_t, tf.transpose(B_t))

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[1, 1, 1],
       [4, 4, 4],
       [9, 9, 9]])>

In [5]:
tf.multiply(A, tf.transpose(B))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [4, 4, 4],
       [9, 9, 9]], dtype=int32)>

In [6]:
tf.less_equal(A, B_t)

<tf.Tensor: shape=(3, 3), dtype=bool, numpy=
array([[ True,  True,  True],
       [False,  True,  True],
       [False, False,  True]])>

In [7]:
C = tf.constant([[1, 1], [2, 2], [3, 3]], shape=[3, 2])

In [8]:
C

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 1],
       [2, 2],
       [3, 3]], dtype=int32)>

In [9]:
for i, row in enumerate(A):
    for j, col in enumerate(tf.transpose(C)):
        added = tf.multiply(row, col)
        val = tf.reduce_sum(added)
        print(f'({i},{j}): {val}')

(0,0): 6
(0,1): 6
(1,0): 12
(1,1): 12
(2,0): 18
(2,1): 18


In [10]:
def representative_dataset():
    for _ in range(100):
      data = np.random.uniform(low=0., high=10., size=(1,8))
      yield [data.astype(np.float32)]
 
tf.config.run_functions_eagerly(True)

forest = RandomForestClassifier(n_estimators=105)
X, y = make_classification(n_samples=1300, n_features=8,
                           n_informative=4, n_redundant=1,
                           random_state=0, shuffle=True,
                           n_classes=4)

x_train, y_train = X[:1000], y[:1000]
x_test, y_test = X[1000:], y[1000:]

forest.fit(x_train, y_train)

X = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int32)
X_float = tf.constant([1., 2., 3., 4., 5., 6., 7., 8.], shape=[1, 8])
X_8 = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int8)

In [11]:
conv_model = convert(forest, 'torch', extra_config={"tree_implementation":"gemm"})

In [12]:
op = conv_model.model._operators[0]

In [13]:
model_gemm = GEMMDecisionTreeImpl(forest)

y_mod_pred_gemm, y_mod_gemm = conv_model.model._operators[0].forward((torch.tensor([[1., 2., 3., 4., 5., 6., 7., 8.]])))

In [14]:
y_mod_gemm

tensor([[0.2571, 0.4952, 0.0476, 0.2000]], grad_fn=<TBackward0>)

In [15]:
y_pred_gemm, y_gemm = model_gemm(X_float)

In [16]:
y_gemm

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.25714287, 0.4952381 , 0.04761905, 0.2       ]], dtype=float32)>

In [17]:
conv_model.model._operators[0].forward((torch.tensor([[5., 5., 1., 2., 1., 8., 7., 3.]])))

(tensor([1]), tensor([[0.2095, 0.4476, 0.0190, 0.3238]], grad_fn=<TBackward0>))

In [18]:
model_gemm(tf.constant([5, 5, 1, 2, 1, 8, 7, 3], shape=[1,8], dtype=tf.float32))

(<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.20952381, 0.44761905, 0.01904762, 0.32380953]], dtype=float32)>)

In [19]:
model_gemm_ne = GEMMDecisionTreeImpl(forest)

In [20]:
concrete_func = model_gemm.__call__.get_concrete_function()

In [21]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model_gemm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8

In [22]:
tflite_model_gemm = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpku51981f/assets
Estimated count of arithmetic ops: 10.202 M  ops, equivalently 5.101 M  MACs


2023-02-09 18:43:32.906563: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-09 18:43:32.906586: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-09 18:43:32.907027: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpku51981f
2023-02-09 18:43:32.919357: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-09 18:43:32.919385: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpku51981f
2023-02-09 18:43:32.919524: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-02-09 18:43:32.969833: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-02-09 18:43:32.970072: I tensorflow/cc/save

Estimated count of arithmetic ops: 10.202 M  ops, equivalently 5.101 M  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
2023-02-09 18:43:34.929986: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1989] Estimated count of arithmetic ops: 10.202 M  ops, equivalently 5.101 M  MACs



In [23]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_gemm)
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  
input = interpreter.get_input_details()[0]  


interpreter.set_tensor(input['index'], X_8)
interpreter.invoke()
y_lite_gemm = interpreter.get_tensor(output['index'])

y_pred_lite_gemm = np.argmax(y_lite_gemm, axis=1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [24]:
y_gemm

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.25714287, 0.4952381 , 0.04761905, 0.2       ]], dtype=float32)>

In [25]:
y_lite_gemm

array([[ 98,  87, 123,  99]], dtype=int8)

In [26]:
y_pred_lite_gemm

array([2])

In [27]:
interpreter.get_tensor(7)

1

In [28]:
with open('../saved_models/random_forest/float32/model.tflite', 'wb') as f:
    f.write(tflite_model_gemm)

# Current stats
### GEMM model Edge TPU
name | value |
|:---------|:---------|
Input model| model.tflite 
Input size | 4.54MiB 
Output model | model_edgetpu.tflite 
Output size | 20.48MiB 
On-chip memory used for caching model parameters | 7.67MiB 
On-chip memory remaining for caching model parameters | 34.25KiB 
Off-chip memory used for streaming uncached model parameters | 9.37MiB 
Number of Edge TPU subgraphs | 4 
Total number of operations | 437 
Number of operations that will run on Edge TPU | 427 
Number of operations that will run on CPU | 10
